# Train Image Similarity

## Mount drive etc

In [121]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Run the Training Script

In [122]:
#pip install torchvision

In [123]:
import torch
from PIL import Image
import os
from tqdm import tqdm
from torch.utils.data import Dataset

import torchvision.transforms as T
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as fn
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [124]:
IMG_PATH = "../data/images"
IMG_HEIGHT = 512  
IMG_WIDTH = 512  

SEED = 42
TRAIN_RATIO = 0.75
VAL_RATIO = 1 - TRAIN_RATIO
SHUFFLE_BUFFER_SIZE = 100

LEARNING_RATE = 1e-3
EPOCHS = 2
TRAIN_BATCH_SIZE = 32  # Let's see, I don't have GPU, Google Colab is best hope
TEST_BATCH_SIZE = 32  # Let's see, I don't have GPU, Google Colab is best hope
FULL_BATCH_SIZE = 32

AUTOENCODER_MODEL_PATH = "../data/models/baseline_autoencoder.pt"
ENCODER_MODEL_PATH = "../data/models/baseline_encoder.pt"
DECODER_MODEL_PATH = "../data/models/baseline_decoder.pt"
EMBEDDING_SHAPE = (1, 64, 64, 64)
# TEST_RATIO = 0.2

In [125]:
class FolderDataset(Dataset):
    def __init__(self, main_dir, transform=None):
        self.main_dir = main_dir
        self.transform = transform
        self.all_imgs = os.listdir(main_dir)

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx):
        #print(self.all_imgs[idx][-3:-1])
        if self.all_imgs[idx][-3:-1] == 'jp': # Only prepare the jpg files
            #print('jpg')
            img_loc = os.path.join(self.main_dir, self.all_imgs[idx])
            image = Image.open(img_loc).convert("RGB")
            resize = fn.resize(image, size=[IMG_WIDTH, IMG_HEIGHT])

            if self.transform is not None:
                tensor_image = self.transform(resize)

            return tensor_image, tensor_image


In [126]:
class ConvEncoder(nn.Module):
    """
    A simple Convolutional Encoder Model
    """

    def __init__(self):
        
        super().__init__()
        # self.img_size = img_size
        self.conv1 = nn.Conv2d(3, 16, (3, 3), padding=(1, 1))
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d((2, 2))

        self.conv2 = nn.Conv2d(16, 32, (3, 3), padding=(1, 1))
        self.relu2 = nn.ReLU(inplace=True)
        self.maxpool2 = nn.MaxPool2d((2, 2))

        self.conv3 = nn.Conv2d(32, 64, (3, 3), padding=(1, 1))
        self.relu3 = nn.ReLU(inplace=True)
        self.maxpool3 = nn.MaxPool2d((2, 2))

        self.conv4 = nn.Conv2d(64, 128, (3, 3), padding=(1, 1))
        self.relu4 = nn.ReLU(inplace=True)
        self.maxpool4 = nn.MaxPool2d((2, 2))

        self.conv5 = nn.Conv2d(128, 256, (3, 3), padding=(1, 1))
        self.relu5 = nn.ReLU(inplace=True)
        self.maxpool5 = nn.MaxPool2d((2, 2))

    def forward(self, x):
        # Downscale the image with conv maxpool etc.
        # print(x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        # print(x.shape)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        # print(x.shape)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)

        # print(x.shape)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.maxpool4(x)

        # print(x.shape)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.maxpool5(x)

        # print(x.shape)
        return x


class ConvDecoder(nn.Module):
    """
    A simple Convolutional Decoder Model
    """

    def __init__(self):
        super().__init__()
        self.deconv1 = nn.ConvTranspose2d(256, 128, (2, 2), stride=(2, 2))
        # self.upsamp1 = nn.UpsamplingBilinear2d(2)
        self.relu1 = nn.ReLU(inplace=True)

        self.deconv2 = nn.ConvTranspose2d(128, 64, (2, 2), stride=(2, 2))
        # self.upsamp1 = nn.UpsamplingBilinear2d(2)
        self.relu2 = nn.ReLU(inplace=True)

        self.deconv3 = nn.ConvTranspose2d(64, 32, (2, 2), stride=(2, 2))
        # self.upsamp1 = nn.UpsamplingBilinear2d(2)
        self.relu3 = nn.ReLU(inplace=True)

        self.deconv4 = nn.ConvTranspose2d(32, 16, (2, 2), stride=(2, 2))
        # self.upsamp1 = nn.UpsamplingBilinear2d(2)
        self.relu4 = nn.ReLU(inplace=True)

        self.deconv5 = nn.ConvTranspose2d(16, 3, (2, 2), stride=(2, 2))
        # self.upsamp1 = nn.UpsamplingBilinear2d(2)
        self.relu5 = nn.ReLU(inplace=True)

    def forward(self, x):
        # print(x.shape)
        x = self.deconv1(x)
        x = self.relu1(x)
        # print(x.shape)

        x = self.deconv2(x)
        x = self.relu2(x)
        # print(x.shape)

        x = self.deconv3(x)
        x = self.relu3(x)
        # print(x.shape)

        x = self.deconv4(x)
        x = self.relu4(x)
        # print(x.shape)

        x = self.deconv5(x)
        x = self.relu5(x)
        # print(x.shape)
        return x


In [127]:
"""
I can write this if we need custom training loop etc.
I usually use this in PyTorch.
"""

__all__ = ["train_step", "val_step", "create_embedding"]

import torch
import torch.nn as nn

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train_step(encoder, decoder, train_loader, loss_fn, optimizer, device):
    # device = "cuda"
    encoder.train()
    decoder.train()

    # print(device)

    for batch_idx, (train_img, target_img) in enumerate(train_loader):
        train_img = train_img.to(device)
        target_img = target_img.to(device)

        optimizer.zero_grad()

        enc_output = encoder(train_img)
        dec_output = decoder(enc_output)

        loss = loss_fn(dec_output, target_img)
        loss.backward()

        optimizer.step()

    return loss.item()


def val_step(encoder, decoder, val_loader, loss_fn, device):
    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        for batch_idx, (train_img, target_img) in enumerate(val_loader):
            train_img = train_img.to(device)
            target_img = target_img.to(device)

            enc_output = encoder(train_img)
            dec_output = decoder(enc_output)

            loss = loss_fn(dec_output, target_img)

    return loss.item()

In [128]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# print("Setting Seed for the run, seed = {}".format(config.SEED))

# seed_everything(config.SEED)

transforms = T.Compose([T.ToTensor()])
print("------------ Creating Dataset ------------")
full_dataset = FolderDataset(IMG_PATH, transforms)
#print(full_dataset.transform)
#print(full_dataset.all_imgs)

train_size = int(TRAIN_RATIO * len(full_dataset))
val_size = len(full_dataset) - train_size
print(train_size)
print(val_size)

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print("------------ Dataset Created ------------")
print("------------ Creating DataLoader ------------")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

#for batch_idx, _ in enumerate(train_loader):
#    print(batch_idx)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=TEST_BATCH_SIZE)

full_loader = torch.utils.data.DataLoader(full_dataset, batch_size=FULL_BATCH_SIZE)

------------ Creating Dataset ------------
4199
1400
------------ Dataset Created ------------
------------ Creating DataLoader ------------


In [129]:
print("------------ Dataloader Cretead ------------")

# print(train_loader)
loss_fn = nn.MSELoss()

encoder = ConvEncoder()
decoder = ConvDecoder()

if torch.cuda.is_available():
    print("GPU Availaible moving models to GPU")
else:
    print("Moving models to CPU")

encoder.to(device)
decoder.to(device)

# print(device)

autoencoder_params = list(encoder.parameters()) + list(decoder.parameters())
optimizer = optim.AdamW(autoencoder_params, lr=LEARNING_RATE)

# early_stopper = utils.EarlyStopping(patience=5, verbose=True, path=)
max_loss = 9999

print("------------ Training started ------------")

for epoch in tqdm(range(EPOCHS)):
    train_loss = train_step(encoder, decoder, train_loader, loss_fn, optimizer, device=device)
    print(f"Epochs = {epoch}, Training Loss : {train_loss}")
    val_loss = val_step(encoder, decoder, val_loader, loss_fn, device=device)

    # Simple Best Model saving
    if val_loss < max_loss:
        print("Validation Loss decreased, saving new best model")
        torch.save(encoder.state_dict(), ENCODER_MODEL_PATH)
        torch.save(decoder.state_dict(), DECODER_MODEL_PATH)
        max_loss = val_loss

    print(f"Epochs = {epoch}, Validation Loss : {val_loss}")

print("Training Done")

------------ Dataloader Cretead ------------
Moving models to CPU
------------ Training started ------------


  0%|          | 0/2 [00:00<?, ?it/s]

Epochs = 0, Training Loss : 0.03257705271244049


 50%|█████     | 1/2 [18:50<18:50, 1130.93s/it]

Validation Loss decreased, saving new best model
Epochs = 0, Validation Loss : 0.035521335899829865
Epochs = 1, Training Loss : 0.01683325134217739


100%|██████████| 2/2 [37:57<00:00, 1138.64s/it]

Validation Loss decreased, saving new best model
Epochs = 1, Validation Loss : 0.016820073127746582
Training Done


In [130]:
embedding_dim = (1, 256, 16, 16)

In [131]:
def create_embedding(encoder, full_loader, embedding_dim, device):
    encoder.eval()
    embedding = torch.randn(embedding_dim)
    # print(embedding.shape)

    with torch.no_grad():
        for batch_idx, (train_img, target_img) in enumerate(full_loader):
            train_img = train_img.to(device)
            enc_output = encoder(train_img).cpu()
            # print(enc_output.shape)
            embedding = torch.cat((embedding, enc_output), 0)
            # print(embedding.shape)
    
    return embedding


In [132]:
embedding = create_embedding(encoder, full_loader, embedding_dim, device)

In [133]:
print(embedding.shape)

torch.Size([5600, 256, 16, 16])


In [134]:
# embedding2 = embedding[4700:, :, :, ]

In [135]:
# print(embedding2.shape)

In [136]:
numpy_embedding = embedding.cpu().detach().numpy()

In [137]:
# numpy_embedding = embedding2.cpu().detach().numpy()

In [138]:
print(numpy_embedding.shape)

(5600, 256, 16, 16)


In [139]:
num_images = numpy_embedding.shape[0]
# print(num_images)

In [140]:
flattened_embedding = numpy_embedding.reshape((num_images, -1))

In [141]:
print(flattened_embedding.shape)

(5600, 65536)


In [142]:
import numpy as np

In [143]:
# np.save("data_embedding_f.npy", flattened_embedding)

In [144]:
np.save("../data/models/data_embedding_f.npy", flattened_embedding)

In [145]:
flattend_embedding_reloaded = np.load("../data/models/data_embedding_f.npy")

In [146]:
# encoder.eval()
# embedding = torch.randn(embedding_dim)
# print(embedding.shape)

# with torch.no_grad():
#     for batch_idx, (train_img, target_img) in enumerate(full_loader):
#         train_img = train_img.to(device)
# #         print(train_img.shape)
        
#         enc_output = encoder(train_img).cpu()
#         print(enc_output.shape)

#         embedding = torch.cat((embedding, enc_output), 0)
#         print(embedding.shape)


# #         break
